# Calidad de los datos de usuarios en distribución

In [1]:
import pandas as pd 
import numpy as np
import unidecode as ud
from pprint import pprint

In [2]:
# Usuarios de la red de Distribución
df_clientes = pd.read_parquet("./NT-Art-1-17-06_2025-v1/2506_CUDx_V2.parquet")

In [3]:
# Se eliminan las columnas que no aportan información relevante
df_clientes = df_clientes.drop(columns=['ID_ClienteKey', 'Cliente_Rut_DV', 'ClienteContacto_CorreoElectronico', 
                                        'ClienteContacto_Telefono', 'ClienteSubsectorEconomico', 'ConexionPunto', 
                                        'ConexionBarra', 'CicloMedicionInicio', 'CicloMedicionTermino', 'ClienteAutorizacion', 
                                        'ClienteNombre_RazonSocial', 'ConexionTipoEmpalme'])

# Se modifican los nombres de columnas para aplicación del .merge
df_clientes = df_clientes.rename(columns={"ConsumoComuna": "Comuna", 
                                          "ID_Distribuidor": "Distribuidor"})

# Se normalizan las columnas Comuna y Distribuidor
for col in ["Comuna", "Distribuidor"]:
    df_clientes[col] = (df_clientes[col].str.lower()                                             # Se vuelve minúscula
                                        .apply(lambda x: ud.unidecode(x) if pd.notna(x) else x)  # Se eliminan acentos
                                        .str.strip())                                            # Se eliminan espacios en blanco
    
# Se reemplaza "cabo de hornos (ex - navarino)" por "cabo de hornos" en la columna "Comuna"
df_clientes["Comuna"] = df_clientes["Comuna"].replace("cabo de hornos (ex - navarino)", "cabo de hornos")

df_clientes    

,Distribuidor,EmpalmeCodigo,ClienteVigencia,ClienteCalidadJuridica,Comuna,ConsumoTipoUsuario,ConsumoTipoConsumo,ConsumoTipoSuministro,ConsumoTipoTarifa,ConsumoTension,ConsumoPotencia,ConsumoFacturado,ConexionAlimentador,ConexionSubestacionPrimariaDistribucion
0,edelmag,101693682,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,44.355,ALIMENTADOR 2 6,PUERTO NATALES
1,edelmag,101693682,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,10.645,ALIMENTADOR 2 6,PUERTO NATALES
2,edelmag,101694022,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,1.936,ALIMENTADOR 2 4,PUERTO NATALES
3,edelmag,101694024,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,4.167,ALIMENTADOR 2 4,PUERTO NATALES
4,edelmag,101836414,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,92.742,ALIMENTADOR 2 2,PUERTO NATALES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7951469,electrica til til,56229,Activo,Persona Natural,tiltil,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,16.0,709.000,RUNGUE,RUNGUE
7951470,electrica til til,56232,Activo,Persona Natural,tiltil,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,3.0,275.000,TILTIL,POLPAICO (ENEL)
7951471,electrica til til,56233,Activo,Persona Natural,tiltil,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,32.0,0.000,TILTIL,POLPAICO (ENEL)
7951472,electrica til til,56244,Activo,Persona Natural,tiltil,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,13kV,15.0,17.000,CALEU,CALEU


### Calidad de datos en la columna: *ConsumoTipoTarifa*

Hay una baja calidad de datos en la columna que representa el tipo de tarifas, partiendo por el hecho de que el df muestra el siguiente conteo de tarifas.

In [4]:
# Conteo de tipos de tarifa normalizado en el DataFrame
df_clientes["ConsumoTipoTarifa"].value_counts(normalize=True, dropna=False).round(3)

ConsumoTipoTarifa
BT1a           0.576
AT2PP          0.234
<NA>           0.148
Peaje_AT3PP    0.022
BT2PP          0.005
AT4.3          0.003
BT4.3          0.002
BT1b           0.002
BT3PP          0.002
BT3PPP         0.002
AT2PPP         0.001
Peaje_AT2PP    0.001
BT2PPP           0.0
AT3PP            0.0
AT3PPP           0.0
Peaje_AT4.3      0.0
BT4.2            0.0
TRAT1            0.0
BT4.1            0.0
AT4.1            0.0
AT4.2            0.0
AT5              0.0
BT5              0.0
TRBT3            0.0
TRAT3            0.0
TRBT2            0.0
Name: proportion, dtype: Float64

In [5]:
# Conteo de tipos de tarifa en el DataFrame
print(df_clientes["ConsumoTipoTarifa"].value_counts())

# 2. Ver cuántos valores nulos había originalmente en la columna de tarifa
print(df_clientes["ConsumoTipoTarifa"].isna().sum())

ConsumoTipoTarifa
BT1a           4580953
AT2PP          1857503
Peaje_AT3PP     177355
BT2PP            41150
AT4.3            23801
BT4.3            18584
BT1b             17562
BT3PP            16604
BT3PPP           13591
AT2PPP            4913
Peaje_AT2PP       4054
BT2PPP            3820
AT3PP             3298
AT3PPP            2851
Peaje_AT4.3       2530
BT4.2             1641
TRAT1             1635
BT4.1              697
AT4.1              496
AT4.2              475
AT5                111
BT5                 68
TRBT3               39
TRAT3               13
TRBT2                3
Name: count, dtype: Int64
1177727


In [7]:
# Tensiones más comunes en la tarifa AT2PP
df_clientes[df_clientes["ConsumoTipoTarifa"] == "AT2PP"]["ConsumoTension"].value_counts().head(20)

ConsumoTension
12 kV     1720945
23kV       128966
0.22kV       2838
0.38kV        874
13kV          150
7kV            90
15kV           37
Name: count, dtype: Int64

In [9]:
# Ver distribución de potencias contratadas en esos AT2PP (kW)
df_clientes[df_clientes["ConsumoTipoTarifa"] == "AT2PP"]["ConsumoPotencia"].describe()

count    1.857503e+06
mean     5.071382e+00
std      3.984463e+00
min      0.000000e+00
25%      2.500000e+00
50%      5.000000e+00
75%      6.000000e+00
max      1.000000e+03
Name: ConsumoPotencia, dtype: float64

In [10]:
# Análisis de calidad de datos en San Miguel
df_san_miguel = df_clientes[df_clientes['Comuna'] == "san miguel"]

# Cantidad de clientes por tipo de tarifa en San Miguel
print(df_san_miguel["ConsumoTipoTarifa"].value_counts())

ConsumoTipoTarifa
AT2PP          58393
Peaje_AT3PP     8138
BT2PP            487
Peaje_AT2PP       62
BT1b              27
BT4.2             26
BT2PPP            14
BT3PP              1
Name: count, dtype: Int64


In [11]:
conteo_tarifas = (
    df_clientes.groupby(["Comuna", "ConsumoTipoTarifa"])
    .size()
    .reset_index(name="Cantidad")
    .sort_values(["Cantidad"], ascending=False)
)
print(conteo_tarifas.head(20))

            Comuna ConsumoTipoTarifa  Cantidad
2236   puente alto              BT1a    194680
2923      santiago             AT2PP    191882
1491         maipu             AT2PP    166430
3275  vina del mar              BT1a    162945
1092    la florida             AT2PP    134838
71     antofagasta              BT1a    132272
1219    las condes             AT2PP    125947
3022        temuco              BT1a    125104
3157    valparaiso              BT1a    123738
2295  punta arenas              BT1a    115757
1721         nunoa             AT2PP    114674
551     concepcion              BT1a    112528
1160     la serena              BT1a    111417
2981         talca              BT1a    110151
624       coquimbo              BT1a    109236
2446      rancagua              BT1a    108298
2660  san bernardo              BT1a    102729
1417   los angeles              BT1a     96895
388        chillan              BT1a     90147
2182   providencia             AT2PP     88821


In [13]:
potencia_stats = (
    df_clientes.groupby(["Comuna", "ConsumoTipoTarifa"])["ConsumoPotencia"]
    .describe()   # cuenta, media, min, max, etc.
    .reset_index()
)

potencia_stats

,Comuna,ConsumoTipoTarifa,count,mean,std,min,25%,50%,75%,max
0,algarrobo,AT2PP,17.0,8.117647,4.662113,6.00,6.000,6.00,6.000,20.00
1,algarrobo,AT2PPP,58.0,7.068966,2.375770,6.00,6.000,6.00,6.000,20.00
2,algarrobo,AT3PP,3.0,42.666667,18.502252,24.00,33.500,43.00,52.000,61.00
3,algarrobo,AT3PPP,20.0,76.400000,89.572317,20.00,24.750,44.00,86.750,381.00
4,algarrobo,AT4.1,2.0,35.000000,21.213203,20.00,27.500,35.00,42.500,50.00
...,...,...,...,...,...,...,...,...,...,...
3340,zapallar,BT3PPP,76.0,28.842368,18.710487,9.50,16.450,26.33,32.910,98.72
3341,zapallar,BT4.1,2.0,10.380000,3.931514,7.60,8.990,10.38,11.770,13.16
3342,zapallar,BT4.3,82.0,29.336585,22.082279,7.60,8.075,26.33,39.490,98.73
3343,zapallar,Peaje_AT4.3,3.0,398.593333,644.779007,26.33,26.330,26.33,584.725,1143.12
